In [1]:
import preprocessData
import calculate
from calculate import Math
from database import bondLengthDatabase, segmentVolumeDatabase
import numpy as np
import sys
import os
import csv
import SGmodel
import resultData

In [163]:
struct = []
f = []
molType = []
allEnergy = []
orgcom = []
spec = []
basises = []
for file in preprocessData.dataName():
    pdgod = preprocessData.getOutputData()
    if file[-3:] == 'out':
        sss = pdgod.dataMolecular(file)
        sss[0], sss[1] = sss[1], sss[0]
        temorg = pdgod.dataOrgcom(file)
        temEnergy = pdgod.dataEnergy(file)
        temStruct = pdgod.dataStructure(file)
        temBasis = pdgod.dataBasis(file)
        temType = pdgod.dataMoltype(file)
        if len(spec) == 0:
            spec.append(sss)
            orgcom.append(temorg)
            allEnergy.append(temEnergy)
            struct.append(temStruct)
            basises.append(temBasis)
            f.append(file)
            molType.append(temType)
        else:
            for i, spe in enumerate(spec):
                if sss[0][2]+sss[1][2] <= spe[0][2]+spe[1][2]:
                    spec.insert(i, sss)
                    orgcom.insert(i, temorg)
                    allEnergy.insert(i, temEnergy)
                    struct.insert(i, temStruct)
                    basises.insert(i, temBasis)
                    f.insert(i, file)
                    molType.insert(i, temType)
                    break       
                try:
                    if sss[0][2]+sss[1][2] > spe[0][2]+spe[1][2] and sss[0][2]+sss[1][2] <= spec[i+1][0][2]+spec[i+1][1][2]:
                        spec.insert(i+1, sss)
                        orgcom.insert(i+1, temorg)
                        allEnergy.insert(i+1, temEnergy)
                        struct.insert(i+1, temStruct)
                        basises.insert(i+1, temBasis)
                        f.insert(i+1, file)
                        molType.insert(i+1, temType)
                        break
                except:pass
                if i == len(spec)-1:
                    spec.append(sss)
                    orgcom.append(temorg)
                    allEnergy.append(temEnergy)
                    struct.append(temStruct)
                    basises.append(temBasis)
                    f.append(file)
                    molType.append(temType)
                    break
    else:continue
print(np.array(f))
print(np.array(orgcom))
print(np.array(spec))


['formaldehyde-formamide_jtz.out' 'formaldehyde-acetamide_jtz.out'
 'propanal-formamide_jtz.out' 'formaldehyde-propanamide_jtz.out'
 'acetaldehyde-acetamide_jtz.out' 'propanal-acetamide_jtz.out'
 'butanal-formamide_jtz.out' 'acetaldehyde-propanamide_jtz.out'
 'propanal-propanamide_jtz.out' 'butanal-acetamide_jtz.out'
 'butanal-propanamide_jtz.out']
[['Amide' 'Aldehyde']
 ['Amide' 'Aldehyde']
 ['Amide' 'Aldehyde']
 ['Amide' 'Aldehyde']
 ['Amide' 'Aldehyde']
 ['Amide' 'Aldehyde']
 ['Amide' 'Aldehyde']
 ['Amide' 'Aldehyde']
 ['Amide' 'Aldehyde']
 ['Amide' 'Aldehyde']
 ['Amide' 'Aldehyde']]
[[[1 0 1 2]
  [1 1 1 3]]

 [[1 0 1 2]
  [1 1 2 5]]

 [[1 0 3 6]
  [1 1 1 3]]

 [[1 0 1 2]
  [1 1 3 7]]

 [[1 0 2 4]
  [1 1 2 5]]

 [[1 0 3 6]
  [1 1 2 5]]

 [[1 0 4 8]
  [1 1 1 3]]

 [[1 0 2 4]
  [1 1 3 7]]

 [[1 0 3 6]
  [1 1 3 7]]

 [[1 0 4 8]
  [1 1 2 5]]

 [[1 0 4 8]
  [1 1 3 7]]]


In [166]:
trainDataName = ['formaldehyde-formamide_jtz.out']

minerr = 1000
for b in range(20, 51):
    excTrainList = []
    for tdn in trainDataName:
        train_index = f.index(tdn)
        inddd = train_index
        print(f[inddd])
        struct3D = struct[inddd]
    #     print(struct3D)
        basis = basises[inddd]
        moltype = molType[inddd]
        fileName = f[inddd]
    #     print(basis)
        referenceEnergy = allEnergy[inddd]
        s3 = calculate.Sort.fragmentSort(struct3D)
        # print(s3[0])
        frag2d, fragStruct3d = [], []
        for s2 in s3:   
            frag, fragStruct = calculate.Convert.fragmentConvert(s2)
            frag2d.append(frag)
            fragStruct3d.append(fragStruct)

        frag2d, fragStruct3d = calculate.Sort.dicectionSort(frag2d, 
                                                            fragStruct3d)
#         print(frag2d, fragStruct3d)

        model_exc = SGmodel.exchange(frag2d, fragStruct3d, basis)
        exclist = model_exc.run(Rphi=3,
                                Rrange=1, beta=b/10, train_mode=True, 
                                referenceExc=referenceEnergy[1])
        print(exclist, '00000000000000000')
        excTrainList.append(exclist[-1][0])
    print('---------------------next---------------------------')
    count = 0
    abserr = 0
    for i, fn in enumerate(f):
        if fn in trainDataName:continue
        print(fn)
        count += 1
        s3 = calculate.Sort.fragmentSort(struct[i])
        frag2d, fragStruct3d = [], []
        for s2 in s3:   
            frag, fragStruct = calculate.Convert.fragmentConvert(s2)
            frag2d.append(frag)
            fragStruct3d.append(fragStruct)
        frag2d, fragStruct3d = calculate.Sort.dicectionSort(frag2d,
                                                            fragStruct3d)

        model_exc = SGmodel.exchange(frag2d, fragStruct3d, basises[i])
        predExc = model_exc.run(Rphi=3, Rrange=1, beta=b/10)
        print(predExc)
        print(allEnergy[i][1])
        if molType[i] == 'Dimer':
            name = fn[:-8] + '-dimer'
        else:
            name = fn[:-8]
        abserr += abs(predExc - allEnergy[i][1])
    aveerr = abserr/count
    if aveerr < minerr:
        minerr = aveerr
        beta = b
    path = './componentDatabase/exc_' + basis + '.csv'
    excComenpentList = []
    with open(path, newline='') as ff:
        rows = csv.reader(ff, delimiter=',')
        rows = np.asarray(list(rows))
        for row in rows:
            if row[0] in excTrainList:
                continue
            excComenpentList.append(row)
    with open(path, 'w', newline='') as fff:
        fff.truncate(0)
        writer = csv.writer(fff)
        writer.writerows(excComenpentList)
    print('====================================================', b)

print(beta, minerr)

formaldehyde-formamide_jtz.out
[[1], [1]]
[['H'], ['F']]
Exc_component:  ['FH'] 
 Exc_component_quantity:  [1] 
 Exc_component_distance:  [[3.2425861650044974]] 

2.0 BBBBB
['FH', 12.386108889120788, 3, 2.0]
[['AA' '0.841951605' '4' '2.7']
 ['BB' '0.544296142' '4' '2.7']
 ['AB' '0.687381624' '4' '2.7']
 ['CC' '3.177775105' '4' '2.7']
 ['AC' '0.65185838' '4' '2.7']
 ['BC' '1.279866763' '4' '2.7']
 ['DD' '5.412029365' '4' '2.7']
 ['AD' '1.083292166' '4' '2.7']
 ['BD' '1.688646013' '4' '2.7']
 ['EE' '13.0434027' '3' '2']
 ['AE' '1.340311202' '4' '2']
 ['BE' '1.815378059' '4' '2']
 ['CE' '4.628852974' '4' '2']
 ['FF' '5.694051935' '3' '2']
 ['CF' '2.779112231' '4' '2']
 ['AF' '1.337290917' '4' '2']
 ['BF' '1.534590415' '4' '2']
 ['GG' '33.25710407' '3' '2']
 ['HH' '26.70445152' '3' '2']
 ['AG' '5.241952601' '3' '2.4']
 ['BG' '7.868931405' '3' '2.4']
 ['CG' '11.27815721' '3' '2.4']
 ['EG' '24.91968282' '3' '2.4']
 ['FG' '17.71752214' '3' '3.8']
 ['AH' '4.151693179' '3' '2']
 ['BH' '5.959795

[[1, -1, 1], [1, -1]]
[['H', 'B', 'A'], ['F', 'A']]
Exc_component:  ['FH'] 
 Exc_component_quantity:  [1] 
 Exc_component_distance:  [[3.655295142970166]] 

3.9687111889425153
9.91863957
propanal-propanamide_jtz.out
[[1, -1, 1], [1, -1, 1]]
[['H', 'B', 'A'], ['F', 'B', 'A']]
Exc_component:  ['FH', 'AH'] 
 Exc_component_quantity:  [1, 1] 
 Exc_component_distance:  [[3.1307365635617828], [3.580584166100602]] 

14.190735460992213
10.66190475
butanal-acetamide_jtz.out
[[-1, 1], [-1, 1, -1, 1]]
[['A', 'H'], ['A', 'B', 'B', 'F']]
Exc_component:  ['BH', 'FH'] 
 Exc_component_quantity:  [1, 1] 
 Exc_component_distance:  [[3.5572469378101106], [3.179097894017039]] 

14.7042163013201
10.6845545
butanal-propanamide_jtz.out
[[1, -1, 1], [1, -1, 1, -1]]
[['H', 'B', 'A'], ['F', 'B', 'B', 'A']]
Exc_component:  ['FH', 'BH'] 
 Exc_component_quantity:  [1, 1] 
 Exc_component_distance:  [[3.1790456507102562], [3.545695927260373]] 

14.759244743445196
10.72007415
==========================================

[[1, -1, 1], [1, -1, 1]]
[['H', 'B', 'A'], ['F', 'B', 'A']]
Exc_component:  ['FH', 'AH'] 
 Exc_component_quantity:  [1, 1] 
 Exc_component_distance:  [[3.1307365635617828], [3.580584166100602]] 

14.856139365588252
10.66190475
butanal-acetamide_jtz.out
[[-1, 1], [-1, 1, -1, 1]]
[['A', 'H'], ['A', 'B', 'B', 'F']]
Exc_component:  ['BH', 'FH'] 
 Exc_component_quantity:  [1, 1] 
 Exc_component_distance:  [[3.5572469378101106], [3.179097894017039]] 

15.093291935898193
10.6845545
butanal-propanamide_jtz.out
[[1, -1, 1], [1, -1, 1, -1]]
[['H', 'B', 'A'], ['F', 'B', 'B', 'A']]
Exc_component:  ['FH', 'BH'] 
 Exc_component_quantity:  [1, 1] 
 Exc_component_distance:  [[3.1790456507102562], [3.545695927260373]] 

15.148628110707008
10.72007415
==================================================== 34
formaldehyde-formamide_jtz.out
[[1], [1]]
[['H'], ['F']]
Exc_component:  ['FH'] 
 Exc_component_quantity:  [1] 
 Exc_component_distance:  [[3.2425861650044974]] 

3.5 BBBBB
['FH', 14.991078275861897, 

[[1, -1, 1], [1]]
[['H', 'B', 'A'], ['F']]
Exc_component:  ['FH'] 
 Exc_component_quantity:  [1] 
 Exc_component_distance:  [[3.2495239907655735]] 

10.10227904561121
10.21413394
acetaldehyde-acetamide_jtz.out
[[-1, 1], [1, -1]]
[['A', 'H'], ['F', 'A']]
Exc_component:  ['FH'] 
 Exc_component_quantity:  [1] 
 Exc_component_distance:  [[3.2532830897416263]] 

9.993517980032628
11.14042105
propanal-acetamide_jtz.out
[[-1, 1], [1, -1, 1]]
[['A', 'H'], ['F', 'B', 'A']]
Exc_component:  ['FH'] 
 Exc_component_quantity:  [1] 
 Exc_component_distance:  [[3.027805053150514]] 

15.86024722342088
10.95719764
butanal-formamide_jtz.out
[[1], [1, -1, 1, -1]]
[['H'], ['F', 'B', 'B', 'A']]
Exc_component:  ['FH', 'BH'] 
 Exc_component_quantity:  [1, 1] 
 Exc_component_distance:  [[3.205118735684017], [3.5521141795271824]] 

14.922117396748439
10.43176689
acetaldehyde-propanamide_jtz.out
[[1, -1, 1], [1, -1]]
[['H', 'B', 'A'], ['F', 'A']]
Exc_component:  ['FH'] 
 Exc_component_quantity:  [1] 
 Exc_compon

[[1], [1]]
[['H'], ['F']]
Exc_component:  ['FH'] 
 Exc_component_quantity:  [1] 
 Exc_component_distance:  [[3.2425861650044974]] 

4.8 BBBBB
['FH', 18.814250526578505, 3, 4.8]
[['AA' '0.841951605' '4' '2.7']
 ['BB' '0.544296142' '4' '2.7']
 ['AB' '0.687381624' '4' '2.7']
 ['CC' '3.177775105' '4' '2.7']
 ['AC' '0.65185838' '4' '2.7']
 ['BC' '1.279866763' '4' '2.7']
 ['DD' '5.412029365' '4' '2.7']
 ['AD' '1.083292166' '4' '2.7']
 ['BD' '1.688646013' '4' '2.7']
 ['EE' '13.0434027' '3' '2']
 ['AE' '1.340311202' '4' '2']
 ['BE' '1.815378059' '4' '2']
 ['CE' '4.628852974' '4' '2']
 ['FF' '5.694051935' '3' '2']
 ['CF' '2.779112231' '4' '2']
 ['AF' '1.337290917' '4' '2']
 ['BF' '1.534590415' '4' '2']
 ['GG' '33.25710407' '3' '2']
 ['HH' '26.70445152' '3' '2']
 ['AG' '5.241952601' '3' '2.4']
 ['BG' '7.868931405' '3' '2.4']
 ['CG' '11.27815721' '3' '2.4']
 ['EG' '24.91968282' '3' '2.4']
 ['FG' '17.71752214' '3' '3.8']
 ['AH' '4.151693179' '3' '2']
 ['BH' '5.959795343' '3' '2']
 ['CH' '8.2433146

In [169]:
for i, fn in enumerate(f):
    print(fn)
    s3 = calculate.Sort.fragmentSort(struct[i])
    frag2d, fragStruct3d = [], []
    for s2 in s3:   
        frag, fragStruct = calculate.Convert.fragmentConvert(s2)
        frag2d.append(frag)
        fragStruct3d.append(fragStruct)
    frag2d, fragStruct3d = calculate.Sort.dicectionSort(frag2d, fragStruct3d)
    print(frag2d)
    print(fragStruct3d)
    model_exc = SGmodel.exchange(frag2d, fragStruct3d, basises[i])
    print('-----------------result---------------')
    predExc = model_exc.run(Rphi=3, Rrange=1, beta=2)
    print(predExc)
    print(allEnergy[i][1])
    if molType[i] == 'Dimer':
        name = fn[:-8] + '-dimer'
    else:
        name = fn[:-8]
    resultData.saveResult(name, 'exc', basises[i], refEnergy=allEnergy[i][1], predEnergy=predExc)

formaldehyde-formamide_jtz.out
[['H'], ['F']]
[[[-1.2853507142857143, -0.13495352380952383, -6.809523809523811e-05]], [[1.9451395714285713, 0.14486342857142856, 8.857142857142859e-05]]]
-----------------result---------------
[[1], [1]]
[['H'], ['F']]
Exc_component:  ['FH'] 
 Exc_component_quantity:  [1] 
 Exc_component_distance:  [[3.2425861650044974]] 

10.30198775
10.30198775
formaldehyde-acetamide_jtz.out
[['A', 'H'], ['F']]
[[[-2.666553, -0.030556, -0.0002], [-0.7152634761904761, -0.11401357142857152, 0.00017666666666666666]], [[2.520709714285714, 0.1344018571428571, -0.00016714285714285716]]]
-----------------result---------------
[[-1, 1], [1]]
[['A', 'H'], ['F']]
Exc_component:  ['FH'] 
 Exc_component_quantity:  [1] 
 Exc_component_distance:  [[3.2454942355268437]] 

10.230003332318166
10.33281132
propanal-formamide_jtz.out
[['H'], ['F', 'B', 'A']]
[[[-2.4646841428571427, 0.180914, -3.666666666666666e-05]], [[0.7339751428571429, -0.4169732857142857, 0.00010285714285714286], [2.3

In [167]:
inddd=0
print(f[inddd])
struct3D = struct[inddd]
print(struct3D)
basis = basises[inddd]
moltype = molType[inddd]
fileName = f[inddd]
print(basis)
referenceEnergy = allEnergy[inddd]
s3 = calculate.Sort.fragmentSort(struct3D)
# print(s3[0])
frag2d, fragStruct3d = [], []
for s2 in s3:   
    frag, fragStruct = calculate.Convert.fragmentConvert(s2)
    frag2d.append(frag)
    fragStruct3d.append(fragStruct)

frag2d, fragStruct3d = calculate.Sort.dicectionSort(frag2d, fragStruct3d)
frag2d, fragStruct3d

formaldehyde-formamide_jtz.out
[[['C', '-1.708901', '-0.116662', '0.00009'], ['O', '-1.152080', '-1.206958', '-0.00022'], ['N', '-1.074617', '1.074516', '-0.00003'], ['H', '-1.606254', '1.925365', '0.00004'], ['H', '-0.059723', '1.106997', '-0.00023'], ['H', '-2.804295', '-0.022634', '0.00009']], [['C', '2.037143', '-0.546216', '0.00010'], ['O', '1.876137', '0.663173', '0.00008'], ['H', '1.180940', '-1.231542', '-0.00006'], ['H', '3.049743', '-0.972249', '0.00028']]]
jtz


([['H'], ['F']],
 [[[-1.2853507142857143, -0.13495352380952383, -6.809523809523811e-05]],
  [[1.9451395714285713, 0.14486342857142856, 8.857142857142859e-05]]])

In [168]:
model_exc = SGmodel.exchange(frag2d, fragStruct3d, basis)
model_exc.run(Rphi=3, Rrange=1, beta=3.8, train_mode=True, referenceExc=referenceEnergy[1])

[[1], [1]]
[['H'], ['F']]
Exc_component:  ['FH'] 
 Exc_component_quantity:  [1] 
 Exc_component_distance:  [[3.2425861650044974]] 

3.8 BBBBB
['FH', 15.698899996847542, 3, 3.8]


array([['AA', '0.841951605', '4', '2.7'],
       ['BB', '0.544296142', '4', '2.7'],
       ['AB', '0.687381624', '4', '2.7'],
       ['CC', '3.177775105', '4', '2.7'],
       ['AC', '0.65185838', '4', '2.7'],
       ['BC', '1.279866763', '4', '2.7'],
       ['DD', '5.412029365', '4', '2.7'],
       ['AD', '1.083292166', '4', '2.7'],
       ['BD', '1.688646013', '4', '2.7'],
       ['EE', '13.0434027', '3', '2'],
       ['AE', '1.340311202', '4', '2'],
       ['BE', '1.815378059', '4', '2'],
       ['CE', '4.628852974', '4', '2'],
       ['FF', '5.694051935', '3', '2'],
       ['CF', '2.779112231', '4', '2'],
       ['AF', '1.337290917', '4', '2'],
       ['BF', '1.534590415', '4', '2'],
       ['GG', '33.25710407', '3', '2'],
       ['HH', '26.70445152', '3', '2'],
       ['AG', '5.241952601', '3', '2.4'],
       ['BG', '7.868931405', '3', '2.4'],
       ['CG', '11.27815721', '3', '2.4'],
       ['EG', '24.91968282', '3', '2.4'],
       ['FG', '17.71752214', '3', '3.8'],
       ['AH', 

In [158]:
model_exc = SGmodel.exchange(frag2d, fragStruct3d, basis)

predExc = model_exc.run()
print(predExc)
print(referenceEnergy[1])

[[1, -1, 1], [1, -1, 1]]
[['A', 'B', 'G'], ['H', 'B', 'A']]
Exc_component:  ['GH'] 
 Exc_component_quantity:  [1] 
 Exc_component_distance:  [[3.0915558775916097]] 

29.926049080182988
29.78896473


In [171]:
preprocessData.dataMove()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\user\\Documents\\research\\SG-model\\molecularStructure\\jtz\\Alkyne-Alkyne\\butyne_jtz.out'